In [2]:
from xgcm import Grid
import pop_tools
import gcsfs
import fsspec as fs
import numpy as np
import xesmf as xe
import xarray as xr
import random
import matplotlib.pyplot as plt
import warnings
from xgcm import Grid
import importlib
import preprocessing
import os

warnings.filterwarnings("ignore")

importlib.reload(preprocessing)
from preprocessing import preprocess_data

In [6]:
heatfluxHR = xr.open_dataset(
    "gs://leap-persistent/panalluri/HF_highRes.zarr", engine="zarr", chunks={}
)
heatfluxHR

<xarray.Dataset>
Dimensions:                        (time: 365, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time                           (time) object 0181-01-01 12:00:00 ... 0181...
  * xt_ocean                       (xt_ocean) float64 -279.9 -279.8 ... 79.95
  * yt_ocean                       (yt_ocean) float64 -81.11 -81.07 ... 89.98
Data variables:
    __xarray_dataarray_variable__  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>

In [4]:
from distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4, threads_per_worker=2)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/panalluri/proxy/8787/status,
Dashboard: /user/panalluri/proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 125.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42741,Workers: 4
Dashboard: /user/panalluri/proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 125.80 GiB
Comm: tcp://127.0.0.1:36723,Total threads: 2
Dashboard: /user/panalluri/proxy/44315/status,Memory: 31.45 GiB
Nanny: tcp://127.0.0.1:37863,


In [11]:
from PIL import Image
import os

data = heatfluxHR.__xarray_dataarray_variable__

# Example plotting function for high-resolution (HR) data only
def plot_fun(idt, vmin, vmax): 
    fig, ax = plt.subplots(figsize=(8, 6))

    # Plot the high-resolution data
    data.isel(time=idt).plot(ax=ax, x='xt_ocean', y='yt_ocean', cmap='bwr', robust=True, vmin=vmin, vmax=vmax, cbar_kwargs={'label': 'Surface Heat Flux (W/m²)'})
    ax.set_title(f'High Res - Frame {idt}')

# Animation creation function remains unchanged
def create_animation(fun, idx, filename='HF_dailyAvg_movie.gif', dpi=200, FPS=18, loop=0):
    # Compute global min and max for the variable
    vmin = data.isel(time=idx).min().compute()
    vmax = data.isel(time=idx).max().compute()
    
    frames = []
    for i in idx:
        fun(i, vmin, vmax)
        plt.savefig('.frame.png', dpi=dpi, bbox_inches='tight')
        plt.close()
        frames.append(Image.open('.frame.png').convert('RGB'))
        print(f'Frame {i} is created', end='\r')
    os.system('rm .frame.png')
    duration = 1000 / FPS
    print(f'Animation at FPS={FPS} will last for {len(idx)/FPS} seconds')
    frames[0].save(
        filename, format='GIF',
        append_images=frames[1:],
        save_all=True,
        duration=duration,
        loop=loop)

In [12]:
# Call the animation creation 
create_animation(plot_fun, range(0, 364, 30), 'HF_dailyAvg_movie.gif', dpi=100, FPS=8)

Animation at FPS=8 will last for 1.625 seconds
